<a href="https://colab.research.google.com/github/aktugchelekche/Amazon_Vine_Analysis/blob/main/S3_ETL_PipeLine_Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [770 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/r

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-16 23:37:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-05-16 23:37:17 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://aktugsbucket.s3.amazonaws.com/user_data.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("user_data.csv"), sep=",", header=True, inferSchema=True)

In [5]:
# Show DataFrame
user_data_df.show(5)

+---+----------+---------+-----------+-------------------+--------------+---------+
| id|first_name|last_name|active_user|     street_address|         state| username|
+---+----------+---------+-----------+-------------------+--------------+---------+
|  1|    Cletus|  Lithcow|      false|78309 Riverside Way|      Virginia|ibearham0|
|  2|       Caz|   Felgat|      false|83 Hazelcrest Place|       Alabama| wwaller1|
|  3|     Kerri|  Crowson|      false|     112 Eliot Pass|North Carolina|ichesnut2|
|  4|   Freddie|    Caghy|      false|    15 Merchant Way|      New York|  tsnarr3|
|  5|   Sadella|    Deuss|      false|   079 Acker Avenue|     Tennessee|fwherrit4|
+---+----------+---------+-----------+-------------------+--------------+---------+
only showing top 5 rows



In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://aktugsbucket.s3.amazonaws.com/user_payment.csv"
spark.sparkContext.addFile(url)
user_payment_df = spark.read.csv(SparkFiles.get("user_payment.csv"), sep=",", header=True, inferSchema=True)

In [7]:
# Show DataFrame
user_payment_df.show(5)

+----------+---------+--------------------+
|billing_id| username|        cc_encrypted|
+----------+---------+--------------------+
|         1|ibearham0|a799fcafe47d7fb19...|
|         2| wwaller1|a799fcafe47d7fb19...|
|         3|ichesnut2|a799fcafe47d7fb19...|
|         4|  tsnarr3|a799fcafe47d7fb19...|
|         5|fwherrit4|a799fcafe47d7fb19...|
+----------+---------+--------------------+
only showing top 5 rows



In [8]:
joined_df=user_data_df.join(user_payment_df , on= "username" , how ="inner")
joined_df.show(5)

+---------+---+----------+---------+-----------+-------------------+--------------+----------+--------------------+
| username| id|first_name|last_name|active_user|     street_address|         state|billing_id|        cc_encrypted|
+---------+---+----------+---------+-----------+-------------------+--------------+----------+--------------------+
|ibearham0|  1|    Cletus|  Lithcow|      false|78309 Riverside Way|      Virginia|         1|a799fcafe47d7fb19...|
| wwaller1|  2|       Caz|   Felgat|      false|83 Hazelcrest Place|       Alabama|         2|a799fcafe47d7fb19...|
|ichesnut2|  3|     Kerri|  Crowson|      false|     112 Eliot Pass|North Carolina|         3|a799fcafe47d7fb19...|
|  tsnarr3|  4|   Freddie|    Caghy|      false|    15 Merchant Way|      New York|         4|a799fcafe47d7fb19...|
|fwherrit4|  5|   Sadella|    Deuss|      false|   079 Acker Avenue|     Tennessee|         5|a799fcafe47d7fb19...|
+---------+---+----------+---------+-----------+-------------------+----

In [9]:
dropna_df = joined_df.dropna()
dropna_df.show(5)

+---------+---+----------+---------+-----------+-------------------+--------------+----------+--------------------+
| username| id|first_name|last_name|active_user|     street_address|         state|billing_id|        cc_encrypted|
+---------+---+----------+---------+-----------+-------------------+--------------+----------+--------------------+
|ibearham0|  1|    Cletus|  Lithcow|      false|78309 Riverside Way|      Virginia|         1|a799fcafe47d7fb19...|
| wwaller1|  2|       Caz|   Felgat|      false|83 Hazelcrest Place|       Alabama|         2|a799fcafe47d7fb19...|
|ichesnut2|  3|     Kerri|  Crowson|      false|     112 Eliot Pass|North Carolina|         3|a799fcafe47d7fb19...|
|  tsnarr3|  4|   Freddie|    Caghy|      false|    15 Merchant Way|      New York|         4|a799fcafe47d7fb19...|
|fwherrit4|  5|   Sadella|    Deuss|      false|   079 Acker Avenue|     Tennessee|         5|a799fcafe47d7fb19...|
+---------+---+----------+---------+-----------+-------------------+----

In [10]:
from pyspark.sql.functions import col

cleaned_df = dropna_df.filter(col("active_user")==True)
cleaned_df.show(5)

+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|  fstappard5|  6|    Fraser|  Korneev|       true|  76084 Novick Court|           Minnesota|         6|a799fcafe47d7fb19...|
|  lhambling6|  7|    Demott|   Rapson|       true|    86320 Dahle Park|District of Columbia|         7|a799fcafe47d7fb19...|
|   wheinerte| 15|   Sadella|    Jaram|       true|7528 Waxwing Terrace|         Connecticut|        15|a799fcafe47d7fb19...|
|droughsedgeg| 17|    Hewitt|  Trammel|       true|    2455 Corry Alley|      North Carolina|        17|a799fcafe47d7fb19...|
|   ydudeniei| 19|       Ted|  Knowlys|       true|      31 South Drive|                Ohio|        19|a799fcafe47d7f

In [11]:
clean_user_df = cleaned_df.select(["id","first_name","last_name" ,"username"])
clean_user_df.show(5)

+---+----------+---------+------------+
| id|first_name|last_name|    username|
+---+----------+---------+------------+
|  6|    Fraser|  Korneev|  fstappard5|
|  7|    Demott|   Rapson|  lhambling6|
| 15|   Sadella|    Jaram|   wheinerte|
| 17|    Hewitt|  Trammel|droughsedgeg|
| 19|       Ted|  Knowlys|   ydudeniei|
+---+----------+---------+------------+
only showing top 5 rows



In [12]:
clean_billing_df = cleaned_df.select(["billing_id" ,"street_address" , "state", "username"])
clean_billing_df.show(5)

+----------+--------------------+--------------------+------------+
|billing_id|      street_address|               state|    username|
+----------+--------------------+--------------------+------------+
|         6|  76084 Novick Court|           Minnesota|  fstappard5|
|         7|    86320 Dahle Park|District of Columbia|  lhambling6|
|        15|7528 Waxwing Terrace|         Connecticut|   wheinerte|
|        17|    2455 Corry Alley|      North Carolina|droughsedgeg|
|        19|      31 South Drive|                Ohio|   ydudeniei|
+----------+--------------------+--------------------+------------+
only showing top 5 rows



In [13]:
clean_payment_df = cleaned_df.select(["billing_id","cc_encrypted"])
clean_payment_df.show(5)

+----------+--------------------+
|billing_id|        cc_encrypted|
+----------+--------------------+
|         6|a799fcafe47d7fb19...|
|         7|a799fcafe47d7fb19...|
|        15|a799fcafe47d7fb19...|
|        17|a799fcafe47d7fb19...|
|        19|a799fcafe47d7fb19...|
+----------+--------------------+
only showing top 5 rows



In [14]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-123.cuamoqmtprvv.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [15]:
# Write DataFrame to active_user table in RDS
clean_user_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)

In [16]:
# Write dataframe to billing_info table in RDS
clean_billing_df.write.jdbc(url=jdbc_url, table='billing_info', mode=mode, properties=config)

In [17]:
# Write dataframe to payment_info table in RDS
clean_payment_df.write.jdbc(url=jdbc_url, table='payment_info', mode=mode, properties=config)